In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
from out_in_bb_backtest import setup_tsl,setup_stop_tp,setup_estr_2
import numpy as np
from backtesting import Strategy, Backtest
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()

ativo = 'EURUSD'
#ativo = 'WIN@N'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 40000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df_puro = df.rename(columns = {'real_volume':'vol'})
df2 = df_puro.copy()

In [2]:
df

,Data,Open,High,Low,Close,Vol
0,2022-12-15 04:48:00,1.06609,1.06615,1.06580,1.06589,0
1,2022-12-15 04:49:00,1.06589,1.06599,1.06587,1.06598,0
2,2022-12-15 04:50:00,1.06598,1.06600,1.06594,1.06600,0
3,2022-12-15 04:51:00,1.06599,1.06604,1.06584,1.06584,0
4,2022-12-15 04:52:00,1.06583,1.06585,1.06574,1.06575,0
...,...,...,...,...,...,...
39995,2023-01-26 02:58:00,1.09155,1.09156,1.09155,1.09155,0
39996,2023-01-26 02:59:00,1.09155,1.09156,1.09151,1.09151,0
39997,2023-01-26 03:00:00,1.09152,1.09154,1.09143,1.09144,0
39998,2023-01-26 03:01:00,1.09144,1.09145,1.09124,1.09124,0


In [3]:
time_period = 14
var_bb = 1.8
pontos = 100
rate_stop = 1
rate_tp = 2
spread = 0.00005
timeframe = 5

new_total = None
for time_period in [14,20,30]:
    for pontos in [0.0003,0.0004,0.0006]:
        for var_bb in [1.8,2,2.2]:
            for rate_tp in [1]:
                df=df_puro.copy()

                bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)
                #bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

                stats = bt.run()


                start = stats['Start'].date()
                end = stats['End'].date()

                n_trades = stats['# Trades']
                taxa_acerto = stats['Win Rate [%]']
                retorno = stats['Return [%]']

                acerto = n_trades * taxa_acerto/100
                erro = n_trades - acerto

                pontos_total = acerto * pontos * rate_tp - erro * pontos * rate_stop - spread * n_trades

                new = pd.DataFrame([[timeframe,pontos,rate_tp,time_period,var_bb,n_trades,taxa_acerto,pontos_total,retorno]])
                new.columns = ['Timeframe','pontos','Rate_tp','Time_period','var_bb','N_trades','Taxa_acerto','Pontos_total','Retorno']
                
                new_total = pd.concat([new_total,new])

                print(new_total.to_string())

   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0003        1           14     1.8       399    48.370927      -0.02385 -0.366086
   Timeframe  pontos  Rate_tp  Time_period  var_bb  N_trades  Taxa_acerto  Pontos_total   Retorno
0          5  0.0003        1           14     1.8       399    48.370927      -0.02385 -0.366086
0          5  0.0003        1           14     2.0       376    49.468085      -0.02000 -0.200345


In [4]:
new_total.sort_values(by = 'Taxa_acerto', ascending=False).iloc[:30]

,Timeframe,pontos,Rate_tp,Time_period,var_bb,N_trades,Taxa_acerto,Pontos_total,Retorno
0,5,0.0004,1,20,2.0,126,63.492063,7.300000e-03,1.224620
0,5,0.0004,1,14,2.0,134,60.447761,4.500000e-03,1.060218
0,5,0.0003,1,14,2.0,134,59.701493,1.100000e-03,0.720400
0,5,0.0004,1,20,1.8,158,59.493671,4.100000e-03,1.060332
0,5,0.0004,1,14,2.2,104,58.653846,2.000000e-03,0.739800
0,5,0.0004,1,20,2.2,94,58.510638,1.700000e-03,0.587921
0,5,0.0003,1,20,2.0,127,58.267717,-5.000000e-05,0.542063
0,5,0.0003,1,14,2.2,105,57.142857,-7.500000e-04,0.455178
0,5,0.0004,1,14,1.8,182,56.593407,5.000000e-04,0.974215
0,5,0.0003,1,14,1.8,184,55.434783,-3.200000e-03,0.531760


In [5]:
new_total.to_excel('eurusd_1M_candlepattern_1.xlsx',index=False)

In [3]:
time_period = 20
var_bb = 1.8
pontos = 0.0006
rate_stop = 1
rate_tp = 1
spread = 5
timeframe = 5

bt = setup_stop_tp(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=True)

stats = bt.run()

#### Setup_Tsl

In [ ]:
time_period = 14
var_bb = 1.8
pontos = 250
rate_stop = 1
rate_tp = 2
spread = 5
timeframe = 5

new_total = None
for time_period in [14,20,30]:
    for var_bb in [1.8,2,2.2]:
        df=df_puro.copy()
        bt = setup_tsl(df,timeperiod=time_period,var_bb=var_bb,pontos = pontos,rate_tp = rate_tp,EMA=False)

        stats = bt.run()


        start = stats['Start'].date()
        end = stats['End'].date()

        n_trades = stats['# Trades']
        taxa_acerto = stats['Win Rate [%]']
        retorno = stats['Return [%]']

        acerto = n_trades * taxa_acerto/100
        erro = n_trades - acerto

        pontos_total = acerto * pontos * rate_tp - erro * pontos * rate_stop - spread * n_trades

        new = pd.DataFrame([[timeframe,pontos,time_period,var_bb,n_trades,taxa_acerto,retorno]])
        new.columns = ['Timeframe','pontos','Time_period','var_bb','N_trades','Taxa_acerto','Retorno']
        
        new_total = pd.concat([new_total,new])

        print(new_total.to_string())

In [ ]:
new_total.to_excel('resultado_win_1M_ema_False_tls.xlsx',index=False)

In [ ]:
bt = setup_tsl(df2,timeperiod=time_period,var_bb=var_bb)

stats = bt.run()

In [ ]:
stats

In [ ]:
bt.plot()

In [ ]:
print(stats['_trades'])